In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import numpy as np

In [3]:
# 0~9 숫자 이미지가 784개의 숫자 (28X28) 로 구성되어 있는 training data 읽어옴
training_data = np.loadtxt('C:/Users/LEEMINJUN/python_study/TensorFlow_Machine_Learning/mnist_train.csv',
 delimiter=',', dtype=np.float32)

# 0~9 숫자 이미지가 784개의 숫자 (28X28) 로 구성되어 있는 test data 읽어옴
test_data = np.loadtxt('C:/Users/LEEMINJUN/python_study/TensorFlow_Machine_Learning/mnist_test.csv',
 delimiter=',', dtype=np.float32)
print("training_data.shape = ", training_data.shape, " ,  test_data.shape = ", test_data.shape)
print("training_data[0,0] = ", training_data[0,0], ",  test_data[0,0] = ", test_data[0,0])
print("len(training_data[0]) = ", len(training_data[0]), ",  len(test_data[0]) = ", len(test_data[0]))

training_data.shape =  (60000, 785)  ,  test_data.shape =  (10000, 785)
training_data[0,0] =  5.0 ,  test_data[0,0] =  7.0
len(training_data[0]) =  785 ,  len(test_data[0]) =  785


In [4]:
train_label_contain, train_image = np.split(training_data,[1],axis = 1)
train_label = np.zeros((len(training_data),10), dtype=float)

test_label_contain, test_image = np.split(test_data,[1],axis = 1)
test_label = np.zeros((len(test_data),10), dtype = float)

for step in range(60000):
    train_label[step,int(train_label_contain[step])] = 1.0
print(train_image.shape, train_label.shape, train_label[0])

for step in range(10000):
    test_label[step, int(test_label_contain[step])] = 1.0
print(test_image.shape, test_label.shape, test_label[0])

(60000, 784) (60000, 10) [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(10000, 784) (10000, 10) [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [24]:
# 입력노드, 은닉노드, 출력노드, 학습율, 반복횟수, 배치 개수 등 설정
learning_rate = 0.3  # 학습율
epochs = 100            # 반복횟수
batch_size = 100      # 한번에 입력으로 주어지는 MNIST 개수

input_nodes = 784     # 입력노드 개수
hidden_nodes = 100    # 은닉노드 개수
output_nodes = 10     # 출력노드 개수

In [25]:
# 입력과 출력을 위한 플레이스홀더 정의
X = tf.placeholder(tf.float32, [None, input_nodes])  
T = tf.placeholder(tf.float32, [None, output_nodes])  

W2 = tf.Variable(tf.random_normal([input_nodes, hidden_nodes]))  # 은닉층 가중치 노드
b2 = tf.Variable(tf.random_normal([hidden_nodes]))               # 은닉층 바이어스 노드

W3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes])) # 출력층 가중치 노드
b3 = tf.Variable(tf.random_normal([output_nodes]))               # 출력층 바이어스 노드

In [26]:
Z2 = tf.matmul(X, W2) + b2    # 선형회귀 선형회귀 값 Z2
A2 = tf.nn.relu(Z2)           # 은닉층 출력 값 A2, sigmoid 대신 relu 사용

# 출력층 선형회귀  값 Z3, 즉 softmax 에 들어가는 입력 값
Z3 = logits = tf.matmul(A2, W3) + b3   

y = A3 = tf.nn.softmax(Z3)

In [27]:
loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T) )#Z3과 T와의 손실함수계산

optimizer = tf.train.GradientDescentOptimizer(learning_rate) # 경사하강법

train = optimizer.minimize(loss)

In [28]:
# batch_size X 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal( tf.argmax(A3, 1), tf.argmax(T, 1) )

# batch_size X 10 의 True, False 를 1 또는 0 으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

In [30]:
import random
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화
    
    for i in range(epochs):    # 100 번 반복수행
        
        total_batch = int(len(training_data) / batch_size)  # 60,000 / 100

        for step in range(total_batch):
            
            #batch_x_data = train_image[i for i in range(100*(step), 100*(step+1))] 안됨
            #batch_t_data = train_label[i for i in range(100*(step), 100*(step+1))]
            
            x_data = []
            t_data = []
            
            for j in range(100*step, 100*(step+1),1): # 100*784, 100*10
                x_data.append(train_image[random.randrange(0,60000)])
                t_data.append(train_label[random.randrange(0,60000)])
                
            batch_x_data = np.array(x_data)
            batch_t_data = np.array(t_data)
            
            batch_x_data = batch_x_data.reshape((100,784))
            batch_t_data = batch_t_data.reshape((100,10))
      
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("epochs = ", i , "step = ", step, ", loss_val = ", loss_val, batch_t_data[0], T)       
    
    # Accuracy 확인
    #test_x_data = mnist.test.images    # 10000 X 784
    #test_t_data = mnist.test.labels    # 10000 X 10
    
    accuracy_val = sess.run(accuracy, feed_dict={X: test_image, T: test_label})
    
    print("\nAccuracy = ", accuracy_val)

epochs =  0 step =  0 , loss_val =  22533.775 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  0 step =  100 , loss_val =  2.2922287 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  0 step =  200 , loss_val =  2.3084533 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  0 step =  300 , loss_val =  2.3038583 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  0 step =  400 , loss_val =  2.3080115 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  0 step =  500 , loss_val =  2.2945807 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  1 step =  0 , loss_val =  2.3067217 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  1 step =  100 , loss_val =  2.3000262 [0. 

epochs =  10 step =  100 , loss_val =  2.293552 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  10 step =  200 , loss_val =  2.3076136 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  10 step =  300 , loss_val =  2.3052208 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  10 step =  400 , loss_val =  2.3043706 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  10 step =  500 , loss_val =  2.305488 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  11 step =  0 , loss_val =  2.2944672 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  11 step =  100 , loss_val =  2.3015194 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  11 step =  200 , loss_val =  2.3015

epochs =  20 step =  100 , loss_val =  2.3054025 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  20 step =  200 , loss_val =  2.3042908 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  20 step =  300 , loss_val =  2.31039 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  20 step =  400 , loss_val =  2.30035 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  20 step =  500 , loss_val =  2.2947443 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  21 step =  0 , loss_val =  2.3045907 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  21 step =  100 , loss_val =  2.3013084 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)
epochs =  21 step =  200 , loss_val =  2.297784

KeyboardInterrupt: 